# Compare SQLs

# Dataset

In [1]:
import pandas as pd
# load csv
db_train = pd.read_csv('llm/data/txt2sql_alerce_train_v4_0.csv')
db_test = pd.read_csv('llm/data/txt2sql_alerce_test_v4_0.csv')

In [2]:
import os
import sqlalchemy as sa
import requests
import time
# Create a connection to the ALeRCE database
def create_conn_alerce(access_time: int = 2):
  """
  Create a connection to the ALeRCE database.
  
  Args:
    access_time: Integer representing the time limit for the connection.
    2 for default access, 10 for extended access.
    
  Returns:
    SQLAlchemy engine object
  
  Raises:
    ValueError: If URL fetch fails or credentials are invalid
  """
  # Common URL for both access levels
  url = "https://raw.githubusercontent.com/alercebroker/usecases/master/alercereaduser_v4.json"
  
  # For security, get extended access credentials from environment variables
  user_extended = os.environ.get('ALERCE_USER_EXTENDED', '')
  pass_extended = os.environ.get('ALERCE_PASS_EXTENDED', '')
  
  n_tries = 3
  params = None
  
  # Fetch parameters from URL with retry logic
  for n_try in range(n_tries):
    try:
      response = requests.get(url)
      if response.status_code != 200:
        if n_try < n_tries - 1:
          time.sleep(2 ** n_try)  # exponential backoff
          continue
        else:
          raise ValueError(f"Failed to fetch URL: {url}, Status Code: {response.status_code}")
      
      params = response.json().get('params')
      if not params:
        raise ValueError("Missing 'params' in the JSON response")
      break
        
    except requests.RequestException as e:
      if n_try < n_tries - 1:
        time.sleep(2 ** n_try)
        continue
      else:
        raise ValueError(f"Network error when fetching {url}: {str(e)}")
    except ValueError as e:
      if "JSON" in str(e):
        raise ValueError("Invalid JSON response from URL")
      else:
        raise e
  
  # Create connection string based on access level
  if access_time == 2:
    conn_string = f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}"
  elif access_time == 10:
    if not user_extended or not pass_extended:
      raise ValueError("Extended access credentials not found in environment variables")
    conn_string = f"postgresql+psycopg2://{user_extended}:{pass_extended}@{params['host']}/{params['dbname']}"
  else:
    raise ValueError(f'Access time {access_time} not supported')
  
  # Create and return engine with connection pooling disabled
  engine = sa.create_engine(conn_string, poolclass=sa.pool.NullPool)
  return engine


def run_sql_alerce(
    sql: str, 
    access_time: int = 2, 
    n_tries: int = 3, 
    query_time: bool = False
):
  ''' Execute the SQL query at the ALeRCE database and return the result
    Args:
        sql: SQL query to execute
        access_time: Integer representing the time limit for the connection. 
        2 for default access, 10 for extended access.
        n_tries: Number of tries to execute the query
        query_time: Boolean indicating whether to track query execution time
    Returns:
    query: pandas dataframe with the result of the query
    error: error message if the query could not be executed
    execution_time: time taken to execute the query    
  '''

  try:
    engine = create_conn_alerce(access_time=access_time)
  except ValueError as e:
    return None, e
    
  query = None
  error = None
  execution_time = None
  
  try:
    for n_try in range(n_tries):
      try:
        with engine.begin() as conn:
          start_time = time.time()
          query = pd.read_sql_query(sa.text(sql), conn)
          if query_time:
            execution_time = time.time() - start_time
          error = None
          break
      except Exception as e:
        error = e
        if n_try < n_tries - 1:
          time.sleep(2 ** n_try)  # exponential backoff
        else:
          # Last attempt failed, keep the error
          pass
  
  finally:
    # Always dispose of the engine to close connections
    engine.dispose()

  if query_time:
    return query, error, execution_time
  else:
    return query, error




In [3]:
gold_columns_train = []
# iter db_train 
n = 5
for indx, row in db_train.iterrows():
    
    query = row['gold_query']
    tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
    if error:
        print(f'Error in query {row.req_id}: {error}')
        gold_columns_train.append({"req_id": row.req_id, "cols": [], "shape": []})
    else:
        print(f'Query {row.req_id} executed successfully, shape: {tables_i.shape}')
        gold_columns_train.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})

Query 13 executed successfully, shape: (4471, 4)
Query 10 executed successfully, shape: (184, 3)
Query 15 executed successfully, shape: (42, 7)
Query 4 executed successfully, shape: (1537, 8)
Query 25 executed successfully, shape: (4, 30)
Query 7 executed successfully, shape: (896, 2)
Query 2 executed successfully, shape: (5732, 2)
Query 3 executed successfully, shape: (11, 6)
Query 23 executed successfully, shape: (668355, 28)
Query 32 executed successfully, shape: (2, 5)
Query 20 executed successfully, shape: (41262, 28)
Query 18 executed successfully, shape: (1041, 11)
Query 35 executed successfully, shape: (10, 6)
Query 11 executed successfully, shape: (491, 4)
Query 9 executed successfully, shape: (54, 8)
Query 5 executed successfully, shape: (781, 9)
Query 0 executed successfully, shape: (8218, 6)
Query 28 executed successfully, shape: (34, 1)
Query 36 executed successfully, shape: (178, 3)
Query 38 executed successfully, shape: (1247, 22)
Query 24 executed successfully, shape: (

In [4]:
gold_columns_test = []
# iter db_test 
n = 5
for indx, row in db_test.iterrows():
    
    query = row['gold_query']
    tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
    if error:
        print(f'Error in query {row.req_id}: {error}')
        gold_columns_test.append({"req_id": row.req_id, "cols": [], "shape": []})
    else:
        print(f'Query {row.req_id} executed successfully, shape: {tables_i.shape}')
        gold_columns_test.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})

Query 8 executed successfully, shape: (186, 28)
Query 27 executed successfully, shape: (5520, 11)
Error in query 17: (psycopg2.errors.QueryCanceled) canceling statement due to statement timeout

[SQL: 
SELECT
  *
FROM
  (
SELECT *
FROM (
SELECT
  feature.oid, feature.value, feature.name, feature.version, feature.fid
FROM
  (
SELECT
    object.oid, object.meanra, object.meandec
FROM
    object INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name='lc_classifier'
    AND probability.class_name = 'AGN'
    AND probability.ranking = 1
) as obj_oids
    INNER JOIN
    feature ON feature.oid = obj_oids.oid
WHERE
  feature.name = 'ExcessVar'
  AND feature.fid = 1
  AND feature.value < -0.001
ORDER BY feature.value ASC
) as sq1
UNION
SELECT *
FROM (
SELECT
  feature.oid, feature.value, feature.name, feature.version, feature.fid
FROM
  (
SELECT
    object.oid, object.meanra, object.meandec
FROM
    object INNER JOIN
    probability
    ON object.oi

## Columns

In [5]:
gold_columns_train

[{'req_id': 13,
  'cols': ['oid', 'class_name', 'lastmjd', 'firstmjd'],
  'shape': (4471, 4)},
 {'req_id': 10, 'cols': ['oid', 'sn_prob', 'cv_prob'], 'shape': (184, 3)},
 {'req_id': 15,
  'cols': ['oid',
   'probability',
   'candid',
   'fid',
   'mjd',
   'magstat_fid',
   'dmdt_first'],
  'shape': (42, 7)},
 {'req_id': 4,
  'cols': ['oid',
   'fid',
   'dmdt_first',
   'candid',
   'f_id',
   'magpsf',
   'sigmapsf_corr',
   'sigmapsf_corr_ext'],
  'shape': (1537, 8)},
 {'req_id': 25,
  'cols': ['oid',
   'ndethist',
   'ncovhist',
   'mjdstarthist',
   'mjdendhist',
   'corrected',
   'stellar',
   'ndet',
   'g_r_max',
   'g_r_max_corr',
   'g_r_mean',
   'g_r_mean_corr',
   'meanra',
   'meandec',
   'sigmara',
   'sigmadec',
   'deltajd',
   'firstmjd',
   'lastmjd',
   'step_id_corr',
   'diffpos',
   'reference_change',
   'oid',
   'classifier_name',
   'class_name',
   'probability',
   'ranking',
   'source_id',
   'ra',
   'dec'],
  'shape': (4, 30)},
 {'req_id': 7, 'cols'

In [6]:
gold_columns_test


[{'req_id': 8,
  'cols': ['oid',
   'ndethist',
   'ncovhist',
   'mjdstarthist',
   'mjdendhist',
   'corrected',
   'stellar',
   'ndet',
   'g_r_max',
   'g_r_max_corr',
   'g_r_mean',
   'g_r_mean_corr',
   'meanra',
   'meandec',
   'sigmara',
   'sigmadec',
   'deltajd',
   'firstmjd',
   'lastmjd',
   'step_id_corr',
   'diffpos',
   'reference_change',
   'oid',
   'classifier_name',
   'classifier_version',
   'class_name',
   'probability',
   'ranking'],
  'shape': (186, 28)},
 {'req_id': 27,
  'cols': ['oid',
   'meanra',
   'meandec',
   'ndet',
   'firstmjd',
   'deltajd',
   'g_r_max',
   'classifier_name',
   'class_name',
   'ranking',
   'probability'],
  'shape': (5520, 11)},
 {'req_id': 17, 'cols': [], 'shape': []},
 {'req_id': 37,
  'cols': ['oid',
   'classifier_name',
   'classifier_version',
   'class_name',
   'probability',
   'ranking'],
  'shape': (200, 6)},
 {'req_id': 34,
  'cols': ['oid', 'name', 'value', 'fid', 'version'],
  'shape': (354, 5)},
 {'req_id